In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv("./local_data/df_final_scaled.csv")

In [460]:
numerical_data = data.filter(data.select_dtypes(["float64", "int64"]).columns)
numerical_data = numerical_data.fillna(numerical_data.median()).dropna(axis=1)
drop_cols = [
    "id",
    "host_id",
    "name",
    "neighbourhood_group",
    "neighbourhood",
    "last_review",
    "scrape_id",
    "price_per_person",
]
numerical_data = numerical_data.drop(
    columns=drop_cols, errors="ignore"
)  # ohne outliers
# numerical_data = numerical_data[
#     numerical_data["price"] < numerical_data["price"].quantile(0.99)
# ]

In [461]:
def transform_features(df: pd.DataFrame) -> pd.DataFrame:
    transformed_df = df.copy()

    numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
    if "price" in numeric_cols:
        numeric_cols.remove("price")

    sqrt_df = df[numeric_cols].apply(lambda x: np.sqrt(x - x.min() + 1))
    sqrt_df.columns = [f"sqrt_{col}" for col in numeric_cols]

    log_df = df[numeric_cols].apply(lambda x: np.log(x - x.min() + 1))
    log_df.columns = [f"log_{col}" for col in numeric_cols]

    # winsorisation
    wins_df = df[numeric_cols].apply(
        lambda x: stats.mstats.winsorize(x, limits=[0.023, 0.023])
    )
    wins_df.columns = [f"wins_{col}" for col in numeric_cols]

    return pd.concat([transformed_df, sqrt_df, log_df, wins_df], axis=1)


numerical_data_transformed = transform_features(numerical_data)

In [462]:
X = numerical_data_transformed.drop("price", axis=1)
y = numerical_data["price"]

In [ ]:
numerical_data_transformed.sort_values("price", ascending=False).head(20)

In [ ]:
y.describe()

In [465]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=99
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [466]:
cv = KFold(n_splits=5, shuffle=True, random_state=99)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rf = RandomForestRegressor()

param_dist = {
    "n_estimators": randint(100, 500),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 10),
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=15,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=99,
)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


# # Define a distribution of parameters to sample from
# param_distributions = {
#     # Continuous range for max_features using uniform distribution
#     'max_features': uniform(0.1, 1.0),

#     # Integer range for number of estimators
#     'n_estimators': randint(50, 500),

#     # Continuous range for minimum samples split
#     'min_samples_split': uniform(0.1, 0.9),

#     # Continuous range for minimum samples leaf
#     'min_samples_leaf': uniform(0.01, 0.2),

#     # Maximum depth with more nuanced sampling
#     'max_depth': [None] + list(randint(10, 50).rvs(10))
# }

# # Create a base random forest regressor
# rf = RandomForestRegressor(random_state=99)

# # Perform randomized search with cross-validation
# random_search = RandomizedSearchCV(
#     estimator=rf,
#     param_distributions=param_distributions,
#     n_iter=100,  # Number of parameter settings that are sampled
#     cv=5,  # 5-fold cross-validation
#     n_jobs=-1,  # Use all available cores
#     verbose=2,  # Show progress
#     random_state=42,  # For reproducibility
#     scoring='neg_mean_squared_error'  # Optimization metric
# )

# # Fit the randomized search to the data
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best Parameters:", random_search.best_params_)

# # Get the best estimator
# best_rf = random_search.best_estimator_

# # Evaluate the best model
# train_score = best_rf.score(X_train, y_train)
# test_score = best_rf.score(X_test, y_test)

# print(f"Train R² Score: {train_score:.4f}")
# print(f"Test R² Score: {test_score:.4f}")

# # Optional: Inspect the results
# results = random_search.cv_results_
# for mean_score, params in zip(results['mean_test_score'], results['params']):
#     print(f"Mean Score: {-mean_score:.4f}, Params: {params}")

In [468]:
# id, accomodates, bathrooms, beds, bedrooms, noise night, noise day

In [476]:
# Random Forest


def perform_rf(X_train_scaled, y_train):
    rf = RandomForestRegressor(
        n_estimators=297,
        random_state=99,
        max_depth=None,
        min_samples_split=6,
        min_samples_leaf=3,
    )

    rf.fit(X_train_scaled, y_train)

    return rf


def evaluate_rf(rf, y_true, y_pred, dataset_name: str):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"{dataset_name} Metrics:")
    print(f"R² Score: {r2:.4f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")

    feature_importance = pd.DataFrame(
        {"feature": X.columns, "importance": rf.feature_importances_}
    ).sort_values(by="importance", ascending=False)

    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "feature_importance": feature_importance,
    }


def visualize_rf_performance(y_true, y_pred, feature_importance):
    fig, axes = plt.subplots(1, 2, figsize=(12, 8))

    # Scatter, actual vs predicted
    axes[0].scatter(y_true, y_pred, alpha=0.5)
    axes[0].plot(
        [y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "r--", lw=2
    )
    axes[0].set_title("Actual vs Predicted Values")
    axes[0].set_xlabel("Actual Values")
    axes[0].set_ylabel("Predicted Values")

    # feature_importance bar chart (top 10)

    top_features = feature_importance.head(10)
    sns.barplot(x="importance", y="feature", data=top_features, ax=axes[1])
    axes[1].set_title("Top 10 Most Important Features")
    axes[1].set_xlabel("Feature Importance")

    plt.tight_layout()
    plt.show()

In [ ]:
# Random forest

rf = perform_rf(X_train_scaled, y_train)  # fit random forest model

y_train_pred_rf = rf.predict(X_train_scaled)  # make predictions
y_test_pred_rf = rf.predict(X_test_scaled)

dict_rf1 = evaluate_rf(
    rf, y_train, y_train_pred_rf, "train"
)  # evaluate with MSE etc. and show most important features

mse1, rmse1, mae1, r21, feature_importance1 = evaluate_rf(
    rf, y_test, y_test_pred_rf, "test"
)

visualize_rf_performance(
    y_train, y_train_pred_rf, dict_rf1["feature_importance"]
)  # create scatter plot with actual vs predicted data, plot bar chart with most important features, or use Y_TEST_PRED_RF TO VISUALISE

In [ ]:
lasso_cv = LassoCV(
    cv=cv,
    random_state=99,
    max_iter=100000,
    alphas=np.logspace(-2, 3, 100),
    selection="random",
    tol=1e-3,
    fit_intercept=True,
)

lasso_cv.fit(X_train_scaled, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(
    lasso_cv.alphas_,
    lasso_cv.mse_path_.mean(axis=1),
    marker="o",
    markersize=3,
    alpha=0.7,
    label="Cross-validation Scores",
)
# Plot the final chosen regularization parameter
ax.vlines(
    lasso_cv.alpha_,
    0,
    ax.get_ylim()[1],
    linestyle="--",
    color="red",
    label=r"Best $\lambda$",
)

# Aesthetics
ax.set_xscale("log")
ax.set_xlabel("Regularization parameter")
ax.set_ylabel("Mean Squared Error")
ax.set_title(r"Lasso Cross-validation Score as a Function of $\lambda$")
ax.grid(alpha=0.3)
ax.legend()

print(f"Best regularization parameter: {lasso_cv.alpha_:.2f}")

In [ ]:
def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray, dataset_name: str) -> None:
    """"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n{dataset_name} Metrics:")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")


y_train_pred = lasso_cv.predict(X_train_scaled)
y_test_pred = lasso_cv.predict(X_test_scaled)

evaluate_model(y_train, y_train_pred, "train")
evaluate_model(y_test, y_test_pred, "test")

In [ ]:
feature_importance = pd.DataFrame({"feat": X.columns, "coeff": lasso_cv.coef_})

feature_importance["Abs_coeff"] = abs(feature_importance["coeff"])
feature_importance_sorted = feature_importance.sort_values("Abs_coeff", ascending=False)
non_zero_features = feature_importance_sorted[feature_importance_sorted["coeff"] != 0]

print(r"\coeffs:")
print(non_zero_features[["feat", "coeff"]].to_string())

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
plt.xlabel("Actual y")
plt.ylabel("Pred y")
plt.title("Actual vs Pred ys (test)")

# Plot 2: feat coeffs
plt.subplot(1, 2, 2)
non_zero_features.plot(kind="bar", x="feat", y="coeff", ax=plt.gca())
plt.xticks(rotation=45, ha="right")
plt.title("coeffs")
plt.tight_layout()
plt.show()

residuals = y_test - y_test_pred

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
sns.histplot(residuals, kde=True)
plt.title("residuals distribuvition")
plt.xlabel("resid val")

plt.subplot(1, 2, 2)
plt.scatter(y_test_pred, residuals, alpha=0.5)
plt.axhline(y=0, color="r", linestyle="--")
plt.xlabel("Pred y")
plt.ylabel("resid")
plt.title("resids vs Pred values")
plt.tight_layout()
plt.show()

In [ ]:
coefficient_df = pd.DataFrame(
    {
        "feature": X.columns,
        "coeff": lasso_cv.coef_,
        "coeff_abs": abs(lasso_cv.coef_),
    }
)

coefficient_df = coefficient_df.sort_values("coeff_abs", ascending=False)
coefficient_df